In [5]:
import backtrader as bt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal

In [2]:
import setup
setup.init_django()
from market import services as market_services

ticker = "NVDA"
days = 720  # Past 2 years

# Fetch 5-minute NVDA data
queryset = market_services.get_5min_stock_quotes_queryset(ticker, days=days)
df_nvda = pd.DataFrame.from_records(list(queryset.values()))

# Convert and clean NVDA data
df_nvda['time'] = pd.to_datetime(df_nvda['time'])
df_nvda.set_index('time', inplace=True)
df_nvda = df_nvda.rename(columns={
    'open_price': 'Open',
    'high_price': 'High',
    'low_price': 'Low',
    'close_price': 'Close',
    'volume': 'Volume'
})
df_nvda = df_nvda[['Open', 'High', 'Low', 'Close', 'Volume']]
df_nvda = df_nvda.tz_localize(None)

In [3]:
sp500_path = '../sp500data/SP500PriceHistory.csv'
df_sp500 = pd.read_csv(sp500_path)

# Convert SP500 dates and create proxy
df_sp500['Date'] = pd.to_datetime(df_sp500['Date'])
df_sp500.set_index('Date', inplace=True)

# Calculate SP500 daily proxy and SMA200
df_sp500['SP500_Close'] = df_sp500.mean(axis=1)
df_sp500['SP500_SMA200'] = df_sp500['SP500_Close'].rolling(200).mean()

# Resample to 5-minute and forward fill
sp500_5min = df_sp500[['SP500_Close', 'SP500_SMA200']].resample('5T').ffill()

# Align dates with NVDA data
start_date = df_nvda.index.min().floor('D')
end_date = df_nvda.index.max().ceil('D')
sp500_5min = sp500_5min[(sp500_5min.index >= start_date) & 
                       (sp500_5min.index <= end_date)]

/tmp/ipykernel_21014/4255442130.py:13: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sp500_5min = df_sp500[['SP500_Close', 'SP500_SMA200']].resample('5T').ffill()


In [6]:
daily_df = df_nvda.resample('D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

daily_df['ma_5'] = daily_df['Close'].rolling(5).mean()
daily_df['ma_20'] = daily_df['Close'].rolling(20).mean()

delta = daily_df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss.replace(0, np.nan)
daily_df['rsi'] = 100 - (100 / (1 + rs))

ema12 = daily_df['Close'].ewm(span=12, adjust=False).mean()
ema26 = daily_df['Close'].ewm(span=26, adjust=False).mean()
daily_df['macd_line'] = ema12 - ema26
daily_df['signal_line'] = daily_df['macd_line'].ewm(span=9, adjust=False).mean()

daily_df['sma_20_bb'] = daily_df['Close'].rolling(20).mean()
daily_df['std_dev'] = daily_df['Close'].rolling(20).std()
daily_df['upper_band'] = daily_df['sma_20_bb'] + 2 * daily_df['std_dev']
daily_df['lower_band'] = daily_df['sma_20_bb'] - 2 * daily_df['std_dev']

daily_df['high_28'] = daily_df['High'].rolling(28).max()
daily_df['low_28'] = daily_df['Low'].rolling(28).min()
daily_df['price_range'] = daily_df['high_28'] - daily_df['low_28']
daily_df['conservative_target'] = daily_df['Close'] + 0.382 * daily_df['price_range']

daily_df['avg_volume_28'] = daily_df['Volume'].rolling(28).mean()
daily_df['volume_change'] = (daily_df['Volume'] - daily_df['avg_volume_28']) / daily_df['avg_volume_28'] * 100

signal_weights = {'ma_crossover':2.0, 'price_target':1.5, 'volume':1.0, 'rsi':1.5, 'macd':1.5, 'bollinger':1.0}
max_score = sum(signal_weights.values())

daily_df['ma_signal'] = np.where(daily_df['ma_5'] > daily_df['ma_20'], signal_weights['ma_crossover'], -signal_weights['ma_crossover'])
daily_df['price_signal'] = np.where(daily_df['Close'] < daily_df['conservative_target'], signal_weights['price_target'], -signal_weights['price_target'])
daily_df['volume_signal'] = np.select(
    [daily_df['volume_change'] > 20, daily_df['volume_change'] < -20],
    [signal_weights['volume'], -signal_weights['volume']],
    default=0
)
daily_df['rsi_signal'] = np.select(
    [daily_df['rsi'] > 70, daily_df['rsi'] < 30],
    [-signal_weights['rsi'], signal_weights['rsi']],
    default=0
)
daily_df['macd_signal'] = np.where(daily_df['macd_line'] > 0, signal_weights['macd'], -signal_weights['macd'])
daily_df['bollinger_signal'] = np.select(
    [daily_df['Close'] < daily_df['lower_band'], daily_df['Close'] > daily_df['upper_band']],
    [signal_weights['bollinger'], -signal_weights['bollinger']],
    default=0
)

daily_df['total_score'] = (daily_df['ma_signal'] + daily_df['price_signal'] + 
                          daily_df['volume_signal'] + daily_df['rsi_signal'] + 
                          daily_df['macd_signal'] + daily_df['bollinger_signal'])
daily_df['normalized_score'] = (daily_df['total_score'] / max_score) * 10
daily_df['recommendation'] = np.select(
    [daily_df['normalized_score'] >= 3, daily_df['normalized_score'] <= -3],
    ['BUY', 'SELL'],
    default='HOLD'
)

daily_indicators = daily_df[['normalized_score', 'recommendation']]
daily_indicators_5min = daily_indicators.resample('5T').ffill()

df_combined = df_nvda.merge(sp500_5min, how='left', left_index=True, right_index=True)
df_combined = df_combined.merge(daily_indicators_5min, how='left', left_index=True, right_index=True)
df_combined.ffill(inplace=True)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'decimal.Decimal'

In [5]:
class MarketAwareData(bt.feeds.PandasData):
    lines = ('sp500_close', 'sp500_sma200', 'indicator_score', 'recommendation')
    params = (
        ('datetime', None),
        ('open', 0),
        ('high', 1),
        ('low', 2),
        ('close', 3),
        ('volume', 4),
        ('sp500_close', 5),
        ('sp500_sma200', 6),
        ('indicator_score', 7),
        ('recommendation', 8)
    )

In [7]:
class IntradayMarketStrategy(bt.Strategy):
    params = (
        ('sma_fast', 20),
        ('sma_slow', 50),
        ('sma_market', 200),
        ('commission', 0.001),
    )

    def __init__(self):
        self.nvda_sma_fast = bt.ind.SMA(self.data.close, period=self.params.sma_fast)
        self.nvda_sma_slow = bt.ind.SMA(self.data.close, period=self.params.sma_slow)
        self.crossover = bt.ind.CrossOver(self.nvda_sma_fast, self.nvda_sma_slow)
        self.market_condition = self.data.sp500_close > self.data.sp500_sma200
        self.last_market_check = None
        
        # Indicator tracking
        self.indicator_score = self.data.indicator_score
        self.recommendation = self.data.recommendation

    def next(self):
        current_date = self.data.datetime.date(0)
        
        if current_date != self.last_market_check:
            self.last_market_check = current_date
            self.current_market_bullish = self.market_condition[0]
            print(f"\n{current_date} - Market Condition: {self.current_market_bullish}")

        # Get current recommendation
        current_recommendation = self.recommendation[0]
        indicator_buy = current_recommendation == 'BUY'
        indicator_sell = current_recommendation == 'SELL'

        # Trading logic
        if not self.position:
            if (self.crossover > 0) and self.current_market_bullish and indicator_buy:
                self.order_target_percent(target=1.0)
                print(f"BUY at {self.data.close[0]:.2f}")
        else:
            if (self.crossover < 0) or not self.current_market_bullish or indicator_sell:
                self.close()
                print(f"SELL at {self.data.close[0]:.2f}")

Initial Portfolio Value: 100000.00

2024-02-21 - Market Condition: Bullish

2024-02-22 - Market Condition: Bullish

2024-02-23 - Market Condition: Bullish

2024-02-24 - Market Condition: Bullish

2024-02-26 - Market Condition: Bullish

2024-02-27 - Market Condition: Bullish

2024-02-28 - Market Condition: Bullish

2024-02-29 - Market Condition: Bullish

2024-03-01 - Market Condition: Bullish

2024-03-02 - Market Condition: Bullish

2024-03-04 - Market Condition: Bullish

2024-03-05 - Market Condition: Bullish

2024-03-06 - Market Condition: Bullish

2024-03-07 - Market Condition: Bullish

2024-03-08 - Market Condition: Bullish

2024-03-09 - Market Condition: Bullish

2024-03-11 - Market Condition: Bullish

2024-03-12 - Market Condition: Bullish

2024-03-13 - Market Condition: Bullish

2024-03-14 - Market Condition: Bullish

2024-03-15 - Market Condition: Bullish

2024-03-18 - Market Condition: Bullish

2024-03-19 - Market Condition: Bullish

2024-03-20 - Market Condition: Bullish

2024

KeyError: 

In [ ]:
cerebro = bt.Cerebro()

# Add data feed
data = MarketAwareData(dataname=df_combined)
cerebro.adddata(data)

# Add strategy
cerebro.addstrategy(IntradayMarketStrategy)

# Configure broker
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.001)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
results = cerebro.run()
strat = results[0]
print("\nFinal Portfolio Value: %.2f" % cerebro.broker.getvalue())

In [ ]:
print("\n--- Strategy Performance ---")
returns_analysis = strat.analyzers.returns.get_analysis()
sharpe_analysis = strat.analyzers.sharpe.get_analysis()
trade_analysis = strat.analyzers.trades.get_analysis()

print(f"Total Return: {returns_analysis.get('rtn100', 0):.2f}%")
print(f"Sharpe Ratio: {sharpe_analysis.get('sharperatio', 0):.2f}")

if trade_analysis.total.closed > 0:
    print(f"\nTrades Closed: {trade_analysis.total.closed}")
    print(f"Win Rate: {trade_analysis.won.total/trade_analysis.total.closed:.2%}")
else:
    print("\nNo trades were executed")

In [ ]:
print("\n--- Data Summary ---")
print(f"NVDA Data Range: {df_nvda.index.min()} to {df_nvda.index.max()}")
print(f"SP500 Data Range: {df_sp500.index.min()} to {df_sp500.index.max()}")
print(f"Combined Data NaNs: {df_combined.isnull().sum().sum()}")

In [ ]:
import matplotlib
%matplotlib inline
matplotlib.rcParams['backend'] = 'module://matplotlib_inline.backend_inline'

cerebro.plot(style='candlestick', 
            iplot=False, 
            volume=True, 
            figsize=(15, 10),
            barup='green', 
            bardown='red',
            subtitles=True,
            plotabove=True,
            overlays=[strat.nvda_sma_fast, strat.nvda_sma_slow])